# Dynamic Programming

Example of Dynamic Programming in Python

## Sample Problem

Suppose you are given two string, $s$ and $t$. Calculate the number of edits need to match these two strings.


### *Example*

|String $s$|String $t$|Edits|
|---|---|---|
|`'a cat!'`|`'the cats!'`|4|
|`'cat'`|`'dog'`|3|
|`'cat'`|`'at'`|1|
|`'cat'`|`'cats'`|1|
|`'cat'`|`'cat`|0|

## What You Can Do

You can do three things for each case:
  1. **Insert** - You can insert a letter into $s$ in the hopes of matching the two strings. 

  > **Important**: This move is equivalent to deleting a letter in $t$.
    
  2. **Substitute** - You can change a letter of $s$ to match it with $t$.
  
  3. **Delete** - You can delete a letter from $t$.

  > **Important**: This move deletes a letter from $s$.

### *Example*

Given string $s$ `'a cat!'` and string $t$ `'the cats!'`. Suppose we start at the end of the two strings when comparing, the following are the principles for the actions:

#### Insert
At the $i$-th iteration where the two strings mismatch, the strings would be `'a cat'` and `'the cats'`. <br>
Inserting at this point will yield the strings `'a cats'` and `'the cats'`, which is essentially equal to `'a cat'` and `'the cat'`.

> **Why?** Why deleting from $t$ instead of keeping the inserted letter? The reason is that it can be ambiguous where to insert in $s$. If the letter should be inserted **before** the $i$-th letter or **after**.

#### Substitute
At the $i$-th iteration where the two strings mismatch, the strings would be `'a cat'` and `'the cats'`. <br>
Substituting at this point will yield the strings `'a cas'` and `'the cats'`, in hopes of matching the two $i$-th letters.

#### Delete
At the $i$-th iteration where the two strings mismatch, the strings would be `'a cat'` and `'the cats'`. <br>
Deleting at this point will yield the strings `'a ca'` and `'the cats'`. 

## Key Notes
It is important to take note that, though, you, as a person, can easily determine which action or operation to do, the computer is not knowledgable of this. In fact, it does not know which of the two strings are lengthier.

It is also important that the strings can get so complex.

## The Algorithm
The algorithm is based off of a paradigm called **Dynamic Programming**, shortened to **DP**. DP essentially optimizes a slow algorithm into a better one. It also tries to find the minimal effort - or a more formal term, **cost** - of a certain algorithm. DP can also be called **Discrete Optimization**.

In [30]:
def countEditDistance(s, t):
  
  def recurse(m, n):
    """
    Return the minimum edit distance between:
      - the first m letters of s
      - the first n letters of t
    """

    if m == 0: # Comparing t to empty s-string
      result = n
    
    elif n == 0: # Comparing s to empty t-string
      result = m
    
    elif s[m - 1] == t[n - 1]: # Both lasts letters are the same
      result = recurse(m - 1, n - 1)
    
    else: # Both strings are mismatch
      insCost = 1 + recurse(m, n - 1)
      subCost = 1 + recurse(m - 1, n - 1)
      delCost = 1 + recurse(m - 1, n)

      result = min(insCost, subCost, delCost)

    return result

  return recurse(len(s), len(t))

> **Note**: Let `m` be the variable to hold the length of $s$ at a certain iteration. We denote the length of $s$ at the $i$-th iteration as $\text{len}({s_i})$. Similarly, we let `n` to hold the length of $t$ at a certain iteration and denote the length of $t$ at the $i$-th iteration as $\text{len}({t_i})$.

## Verifying The Code

Let's now check if the code does truly work. 

In [31]:
s = 'a cat!'
t = 'the cats!'

print(countEditDistance(s, t))

4


Yehey! So the code works perfectly. But, what if the length of the string is a bit longer? ...

Try to run the code below and see if it is as fast as the previous strings $s$ and $t$.

In [18]:
import time

s = 'a cat!' * 2 # This will become 'a cat!a cat!a cat!...' 2 times of the string 'a cat!'
t = 'the cats!' * 2

start = time.time()
print('Count distance: ', countEditDistance(s, t))
end = time.time()

print('Elapsed time: ', end - start) # Printing the elapsed time 

Count distance:  8
Elapsed time:  13.710598230361938


> Even with just double the length of the samples, the runtime of the algorithm drastically increased.

To analyze the time complexity of the `countEditDistance` function, we need to carefully examine the recursive `recurse` function it contains. Here’s a step-by-step analysis of the time complexity:

### Step-by-Step Analysis
1. **Recursive Calls**
  - Each `recurse(m, n)` call generates three (3) recursive calls in the worst case. (One for each edit operation).

2. **Exponential Number of Calls**
  - The total number of calls given by $T(m, n)$:
  $$
  T(m, n) = 3 \cdot T(m - 1, n) + 3 \cdot T(m, n - 1) + 3 \cdot T(m - 1, n - 1)
  $$

  - This is an upper bound approximation because the actual recursion tree may have fewer branches due to the base cases and character matches.

3. **Approximate Complexity**
  - The recurrence relation resembles that of an exponential function. The number of calls grows exponentially with respect to $m$ and $n$.
  - The overall time complexity can be approximated as:
  $$
  T(m, n) = O(3^{\max{(m, n)}})
  $$

  - Since each subproblem can lead to three further subproblems and each level of the tree has a branching factor of approximately three (3).

## Optimizing the Algorithm

How do we optimize this? 

In dynamic programming, algorithms are usually optimized by storing or remembering the results that could have been evaluated midway during solving. The formal term used by computer scientists for this optimization is called **memoization**.

### Theory of How It Works

At some point, during the execution of the algorithm, the branches could have - at some point - reached a phenomenon where the nodes are identical or have already been evaluated. 

Consider this part of the algorithm:

```python
  insCost = 1 + recurse(m, n - 1)
  subCost = 1 + recurse(m - 1, n - 1)
  delCost = 1 + recurse(m - 1, n)
```

In each of the recursive branch of these cases, there could be a case where a similar string length that has already been evaluated before. Let's dive into coding the memoization function.

> *Note*: It is said that it is better to come up with a *slow* algorithm first before improving it to make it fast and not being slick with it immediately.

In [23]:
def countEditDistanceMemoize(s, t):
  cache: dict = {} # This is the storage for the already-evaluated cases, our memoization container

  # Same code with 3-4 lines of changes in the code
  def recurse(m, n):
    """
    Return the minimum edit distance between:
      - the first m letters of s
      - the first n letters of t
    """

    if (m, n) in cache:
       return cache[(m, n)]

    if m == 0: # Comparing t to empty s-string
      result = n
    
    elif n == 0: # Comparing s to empty t-string
      result = m
    
    elif s[m - 1] == t[n - 1]: # Both lasts letters are the same
      result = recurse(m - 1, n - 1)
    
    else: # Both strings are mismatch
      insCost = 1 + recurse(m, n - 1)
      subCost = 1 + recurse(m - 1, n - 1)
      delCost = 1 + recurse(m - 1, n)

      result = min(insCost, subCost, delCost)

    cache[(m, n)] = result
    return result

  return recurse(len(s), len(t))

> After the changes in the code at lines 2, 12, 13, and 31, our algorithm will now work tremendously faster than the previous algorithm. Let's try it.

In [27]:
s = 'a cat!' * 2 # This will become 'a cat!a cat!a cat!...' 2 times of the string 'a cat!'
t = 'the cats!' * 2

start = time.time()
print('Count distance: ', countEditDistanceMemoize(s, t))
end = time.time()

print('Elapsed time: ', end - start) # Printing the elapsed time 

print('---')

s = 'a cat!' * 10 # This will become 'a cat!a cat!a cat!...' 10 times of the string 'a cat!'
t = 'the cats!' * 10

start = time.time()
print('Count distance: ', countEditDistanceMemoize(s, t))
end = time.time()

print('Elapsed time: ', end - start) # Printing the elapsed time 

Count distance:  8
Elapsed time:  0.0009996891021728516
---
Count distance:  40
Elapsed time:  0.01061391830444336


> Wow, even at a string length of 60, the algorithm performs faster than the previous one of just double the length!

## Analysis

**What happened?**

The `cache` held the values of the existent $m$ and $n$ lengths since there is no need to re-evaluate this length. <br>
Note that we do not need to care if at $\text{len}(s_i)$ and $\text{len}(t_i)$ , the letters are different from $\text{len}(s_j)$  and $\text{len}(t_j)$ . This is because we will just either way recurse it to the three edit operations.<br>
Also note that, we primarily check for the cache for existing `m` and `n` and immediately return out of the function. If there is no existing `m` and `n` in the cache, it will be appended at the end before returning the result.

## Conclusion

The time complexity of the `countEditDistance` function is exponential, specifically $O(3^{\max{(m, n)}})$. This is due to the recursive nature of the function where each subproblem generates up to three more subproblems without memoization to avoid redundant calculations. In practical applications, this approach is inefficient for large strings and would benefit greatly from dynamic programming with memoization to reduce the time complexity to polynomial time $O(m \times n)$.